In [0]:

import yaml


yaml_path = "/Workspace/Users/tushhaar24@gmail.com/Databricks_flight_project/Databricks_Flight_Data_Project/configs/autoloader_bronze.yml"

with open(yaml_path, "r") as f:
    config = yaml.safe_load(f)

autoloader_cfg = config["autoloader"]


file_format = autoloader_cfg["file_format"]
options = autoloader_cfg.get("options", {})
checkpoint_template = autoloader_cfg["checkpoint_location"]
target_template = autoloader_cfg["target_path"]
source_template = autoloader_cfg["source_path"]
tables = autoloader_cfg["tables"]


dbutils.widgets.text("src", "")
src_value = dbutils.widgets.get("src")


if src_value and src_value in tables:
    tables_to_process = [src_value]
else:
    tables_to_process = tables

print(f"Tables selected for ingestion: {tables_to_process}")


for table in tables_to_process:
    print(f"Processing table: {table}")

    
    checkpoint_path = checkpoint_template.format(table=table)
    target_path = target_template.format(table=table)
    source_path = source_template.format(table=table)

    
    df = (
        spark.readStream.format("cloudFiles")
        .option("cloudFiles.format", file_format)
        .options(**options)  
        .option("cloudFiles.schemaLocation", checkpoint_path)
        .load(source_path)
    )

    
    (
        df.writeStream.format("delta")
        .outputMode("append")
        .trigger(once=True)
        .option("checkpointLocation", checkpoint_path)
        .option("path", target_path)
        .start()
    )

print("Bronze ingestion job submitted.")
